In [1]:
import os
import json
import re
import string
from pprint import pprint
from pathlib import Path
from collections import defaultdict

from dotenv import load_dotenv
from langchain.llms import OpenAI

from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory



In [2]:
load_dotenv()

True

In [3]:
DATAROOT = "data"

In [7]:
def read_json_file(json_file_path):
    try:
        with open(json_file_path, 'r') as json_file:
            data = json.load(json_file)
        return data
    except FileNotFoundError:
        print(f"File not found: {json_file_path}")
        return None
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {str(e)}")
        return None

def load_data(root_path: str):
    """Load all word data """
    data = []
    for fn in os.listdir(root_path):
        data.extend(read_json_file(root_path + "/" + fn))
    
    return data

In [8]:
word_schatz = load_data(DATAROOT)

In [12]:
pprint(word_schatz[0])

{'überstanden': {'english': ['endured'],
                 'sentences': ['Das Winter überstanden.',
                               'Sie haben den Sturm überstanden.',
                               'Wir werden diese Krise überstehen.',
                               'Wenn ich den Marathon beendet habe, habe ich '
                               'den Lauf überstanden.',
                               'Es wird erwartet, dass sie die Prüfung '
                               'bestehen werden.'],
                 'synonyms': {'durchkommen': {'english': ['come through'],
                                              'sentences': ['Er kam durch die '
                                                            'schwierige '
                                                            'Prüfung.',
                                                            'Die Mannschaft '
                                                            'kam erfolgreich '
                                               

In [29]:
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory



prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("""
        You are a friendly AI and a German tutor that creates challenges to be solved inorder to help the human improve their German language skills.
        Generate a challenge given a german word, and a mode. A mode will be one of the following:
        1. Präsens
        2. Perfekt
        3. Plusquamperfekt
        4. Futur1
        5. Futur2
        6. Subjunctiv I
        7. Subjunctive II
        8. PartizipII
        The challenge will be generated in english. Give the student your feed back on their performance.
        Create the challenge once you see the token <Start>
        """                               
    ),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("""
        Examples:
            Mode: Präsens
            Word: Beweisen    
            Challenge: How would you say in German: "I prove my innocence"
            Student Response: Ich beweise meine Unschuld
            Tutor Feedback: Great work!

            Mode: Subjunctive II
            Word: Beweisen    
            Challenge: How would a German say: "If he had proven his theory, he would have become famous."
            Student Response:  Wenn er seine Theorie bewiesen hätte, wäre er berühmt geworden
            Tutor Feedback: Spot on!

            Mode: Perfekt
            Word: Beweisen    
            Challenge: Your German professor has just lectured you on origins on the universe. He gave you reasons why he feels evolution is not true. 
            Tell your german friend: "He has proven his theory"
            Student Response:  Sie hat seine Theorie bewiesen
            Tutor Feedback: You were close, but Sie is used for a lady, the correct Pronoun is Er.

        <Start>:
            Mode: FuturI
            Word:  {the_word}    
    """)
])

llm = ChatOpenAI(temperature=0)
memory = ConversationBufferMemory(return_messages=True)
conversation = LLMChain(memory=memory, prompt=prompt, llm=llm)

In [32]:
response = conversation(inputs={"the_word":"überstanden"})
print(response["text"])

Challenge: Imagine you are talking to your friend about a difficult situation you are currently facing. Tell your friend: "We will have overcome this challenge."

Student Response: Wir werden diese Herausforderung überstanden haben.

Tutor Feedback: Excellent job! Your sentence is grammatically correct and conveys the intended meaning. Well done!


In [39]:
from langchain.prompts import (
    FewShotChatMessagePromptTemplate,
    ChatPromptTemplate,
)

examples = [
    {"another_input": "another input 1", "input": "2+2", "output": "4"},
    {"another_input": "another input 2", "input": "2+3", "output": "5"},
]

# This is a prompt template used to format each individual example.
example_prompt = ChatPromptTemplate.from_messages(
    [   
        ("human", "{another_input} : {input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt_template = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

print(few_shot_prompt_template.format())

Human: another input 1 : 2+2
AI: 4
Human: another input 2 : 2+3
AI: 5


In [44]:
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts import (FewShotChatMessagePromptTemplate,
                                 MessagesPlaceholder,
                                ChatPromptTemplate)

# system prompt

system_prompt = """
    You are a friendly AI and a German tutor that creates challenges to be solved inorder to help the human improve their German language skills.
    Given a challenge, and your student's response, you give helpful feedback to guide the student to the right solution.
    You give all your feedback in English translating any English word in the challenge to German.
"""

system_prompt_template = ChatPromptTemplate.from_messages([("system", system_prompt)])

# few shot promt

examples = [
    {"challenge": "A German driver is about to approach your bus stop, tell him in German: I am alighting", 
     "student_response": "Ich steige aus", 
     "tutor_feedback": "Bravo! Seems you're really gonna be fluent soon"},

    {"challenge": "How would your German friend tell you, We are free now!", 
     "student_response": "Sie sind frei!", 
     "tutor_feedback": "Good try but the article used stands for You. Try to use the article for We"},

    {"challenge": "How would your German friend tell you, We are free now!", 
     "student_response": "Wir sind frei!", 
     "tutor_feedback": "Great work! But there is just a little bit left."},

    {"challenge": "How would your German friend tell you, We are free now!", 
     "student_response": "Wir sind jetzt frei!", 
     "tutor_feedback": "Excellent! Keep on practicing your German skills"},
]

# This is a prompt template used to format each individual example.
example_prompt_template = ChatPromptTemplate.from_messages(
    [   
        ("human", """Challenge: {challenge}
                    Student's Response: {student_response}
                 """),
        ("ai", "Tutor's Feedback: {tutor_feedback}"),
    ]
)
few_shot_prompt_template = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt_template,
    examples=examples,
)


challenge_prompt_template = ChatPromptTemplate.from_messages(
    [   
        ("human", "{input}")
    ]
)

In [45]:
full_prompt = """
    {system_prompt}
    {history}
    {few_shot_prompt}
    {input}
"""
full_prompt_template = PromptTemplate.from_template(full_prompt)

input_prompts = [
    ("system_prompt", system_prompt_template),
    ("few_shot_prompt", few_shot_prompt_template),
]

pipeline_prompt = PipelinePromptTemplate(final_prompt=full_prompt_template, pipeline_prompts=input_prompts)
pipeline_prompt.format(input="this is input", history="this is history")

'\n    [SystemMessage(content="\\n    You are a friendly AI and a German tutor that creates challenges to be solved inorder to help the human improve their German language skills.\\n    Given a challenge, and your student\'s response, you give helpful feedback to guide the student to the right solution.\\n    You give all your feedback in English translating any English word in the challenge to German.\\n", additional_kwargs={})]\n    this is history\n    [HumanMessage(content="Challenge: A German driver is about to approach your bus stop, tell him in German: I am alighting\\n                    Student\'s Response: Ich steige aus\\n                 ", additional_kwargs={}, example=False), AIMessage(content="Tutor\'s Feedback: Bravo! Seems you\'re really gonna be fluent soon", additional_kwargs={}, example=False), HumanMessage(content="Challenge: How would your German friend tell you, We are free now!\\n                    Student\'s Response: Sie sind frei!\\n                 ", addit

In [49]:
llm = ChatOpenAI(temperature=0)
memory = memory=ConversationBufferMemory(
            memory_key="history",
            input_key="human_input"
        )
conversation_chain = ConversationChain(
    llm=llm, 
    verbose=True,
    prompt=pipeline_prompt,
)
conversation_chain.predict(input="""Challenge: How would you tell your German girl: I love you
                        Student's Response: Ich liebe dir
                    """)

ValidationError: 1 validation error for ConversationChain
__root__
  Got unexpected prompt input variables. The prompt expects [], but got ['history'] as inputs from memory, and input as the normal input key. (type=value_error)